This is a notebook that I have created to test the various features of the BaconBits cape.

I received the PocketBeagle when I enrolled in the [e-ale course](https://e-ale.org/) at the 2018 [Embedded Linux Conference](https://events.linuxfoundation.org/events/elc-openiot-north-america-2018/).

The resources for this course at contained on [github](https://github.com/e-ale).

The code that I am using to initially test the features of the BaconBits cape is located [here](https://github.com/e-ale/Code/tree/master/RESOURCES/pocketbeagle).

## Wifi setup

In order to easily use this notebook, I first need to get the PocketBeagle hooked up to the internet.

To do this, on my laptop, I type:
```
sudo su
iptables --table nat --append POSTROUTING --out-interface wlp3s0 -j MASQUERADE
iptables --append FORWARD --in-interface enx38d269566e5d -j ACCEPT
echo 1 > /proc/sys/net/ipv4/ip_forward
exit
```
and then on the PocketBeagle I type:
```
sudo ip route add default via 192.168.7.1
echo "nameserver 8.8.8.8" | sudo tee /etc/resolv.conf
```

I would like to not have to run the above commands after every reboot of the PocketBeagle, but at the moment, I don't know had to do that.

I can get around the resolv.conf issue by disabling connman:
```
sudo systemctl disable connman.service
```
but can not figure out how to get the default gateway working on boot, despite having configure `/etc/network/interfaces` to:
```
iface usb0 inet static
    address 192.168.7.2
    netmask 255.255.255.0
    gateway 192.168.7.1
    dns-nameservers 8.8.8.8
```

I then cloned the git repository using:
```
git clone https://github.com/b-trav/BaconBitsIntro.git
```

Initially when I set up git I did these commands to set my username, email, and to allow me to push without typing my github password:
```
git config --global user.name "Ben Travaglione"
git config --global user.email "ben@travaglione.com"
git config --global credential.helper store
```

## Jupyter

Evidently, I am writing this notebook in Jupyter, and the notebook is designed to be run on the PocketBeagle, thus you need to get Jupyter up and running on the PocketBeagle.

This is accomplished with the following commands:

```
sudo -H pip install --upgrade pip
sudo -H pip install jupyter
jupyter notebook --generate-config
```
I then allow external access to the notebook server by editing the jupyter config file:
```
vim .jupyter/jupyter_notebook_config.py
```
and uncommenting
```
c.NotebookApp.ip = '*'
```
and
```
c.NotebookApp.allow_root = True
```

I also need to add password access to the notebook server:
```
jupyter notebook password
```
I allow jupyter to boot as root at startup by adding:
```
@reboot /usr/bin/python /usr/local/bin/jupyter-notebook --config /home/debian/.jupyter/jupyter_notebook_config.py --notebook-dir /home/debian/
```
to my crontab by typing
```
sudo crontab -e
```

These days I find Jupyter annoying to use without installing extensions, so these are the commands to install some Jupyter extensions:
```
sudo apt install python-dev libxml2-dev libxslt1-dev zlib1g-dev
sudo -H pip install jupyter_contrib_nbextensions # This command is not working!
```
At the moment I can not install the extensions. For some reason libxml is not compiling.
```
jupyter contrib nbextension install --user
jupyter nbextension enable codefolding/main
jupyter nbextension enable freeze/main
jupyter nbextension enable spellchecker/main
jupyter nbextension enable toggle_all_line_numbers/main
jupyter nbextension enable collapsible_headings/main
jupyter nbextension enable init_cell/main
jupyter nbextension enable scratchpad/main
jupyter nbextension enable toc2/main

sudo -H pip install jupyter_nbextensions_configurator
jupyter nbextensions_configurator enable --user
```

## Potentiometer

Let's look at the potentiometer on the BaconBits.

In bash, we can grab the value of the potentiometer very easily:

In [ ]:
f = '/sys/bus/iio/devices/iio\:device0/in_voltage0_raw'
!ls $f
!cat $f


This can be a bit trickier in python because it does not seem to like the filename for some reason:

In [ ]:
with open(f,'r') as pot:
    val = pot.readline()
    print(val)

I can get around this problem by creating a symbolic link:

In [ ]:
!rm potentiometer
!ln -s /sys/bus/iio/devices/iio\:device0/in_voltage0_raw potentiometer

In [ ]:
lnf = 'potentiometer'
with open(lnf,'r') as pot:
    val = pot.readline()
    print(val)

But this seems like a bit of a hack!

# User Button

In bash, we can get the status of the user button with:

In [ ]:
!cat /sys/class/gpio/gpio45/value

In [ ]:
from IPython.display import clear_output

button_filename = '/sys/class/gpio/gpio45/value'
with open(button_filename,'r') as button_file:
    val = button_file.readline()
last_val = val
print("Current User-button value: {} (Interrupt the kernel to quit)".format(val))
while True:
    with open(button_filename,'r') as button_file:
        val = button_file.readline()
    if not (val == last_val):
        last_val = val
        clear_output()
        print("Current User-button value: {} (Interrupt the kernel to quit)".format(val))
        


# SPI 7-segment LEDs

In bash, we can turn on the 7-segment LEDs buy typing:
```
echo -ne '\x40\x01\x02' | sudo tee /dev/spidev2.1 | xxd
```
or, because this jupyter is running as root:

In [ ]:
import binascii
hexstr = '400102'
binstr = binascii.unhexlify(hexstr)
!echo $binstr >> /dev/spidev2.1

In python, we can do the following:

In [ ]:
import binascii
hexstr = '400000'
binstr = binascii.unhexlify(hexstr)

with open('/dev/spidev2.1','wb') as f:
    f.write(binstr)

Now let's try and figure out how that hex string translates into the pattern on the LED display:

In [ ]:
hexstr = '4000FF'
binstr = binascii.unhexlify(hexstr)

with open('/dev/spidev2.1','wb') as f:
    f.write(binstr)

In [ ]:
import time

reg = 4 << 20
left = 0
right = 1 << 8
for i in range(0,7):
    value = reg + i + left
    hexstr = "{:X}".format(value)
    print(hexstr)
    binstr = binascii.unhexlify(hexstr)

    with open('/dev/spidev2.1','wb') as f:
        f.write(binstr)
    time.sleep(.2)

Let's make some code which makes the lights go around in a circle


In [3]:
import time
import binascii

delay = 0.05
num_loops = 10

def led(side,state):
    # Change the state of the 7-segment LEDs on the BaconBits cape
    
    if state == 'off':
        state = 127
    if state == 'on':
        state = 0
    
    reg = 4 << 20
    value = reg + (side << 8) + state
    hexstr = "{:X}".format(value)
    binstr = binascii.unhexlify(hexstr) 
    with open('/dev/spidev2.1','wb') as f:
        f.write(binstr)

for i in range(num_loops):
    for k in range(6):
        led(0,127-2**k)
        led(1,127-2**k)
        time.sleep(delay)
        
led(0,'off')
led(1,'off')

In [ ]:
led(0,'off')

Now let's see if we can get the lights to spin in opposite directions:

In [ ]:
for i in range(num_loops):
    for k in range(6):
        led(0,127-2**k)
        led(1,127-2**(5-k))
        time.sleep(delay)
led(0,'off')
led(1,'off')

Now let's get an interactive plot happening.

The BaconBits Rev.1 image comes from https://oshpark.com/profiles/mwelling.

In [4]:
import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

p = figure(x_range=(0,1086), y_range=(0,692),width=1086/2,height=692/2)
#p = figure()
p.image_url(url=['BaconBits.png'], x=0, y=692,w=1086,h=692,angle=0)
led_h = [[],[]]

led_h[1].append(p.rect(x=[480],y=[434],width=20,height=40,color='yellow',fill_alpha=0)) # 0
led_h[1].append(p.rect(x=[504],y=[473],width=40,height=20,color='yellow',fill_alpha=0)) # 1
led_h[1].append(p.rect(x=[565],y=[473],width=40,height=20,color='yellow',fill_alpha=0)) # 2
led_h[1].append(p.rect(x=[591],y=[434],width=20,height=40,color='yellow',fill_alpha=0)) # 3
led_h[1].append(p.rect(x=[504],y=[388],width=40,height=20,color='yellow',fill_alpha=0)) # 4
led_h[1].append(p.rect(x=[565],y=[388],width=40,height=20,color='yellow',fill_alpha=0)) # 5
led_h[1].append(p.rect(x=[536],y=[434],width=20,height=40,color='yellow',fill_alpha=0)) # 6

diff = 494-324
led_h[0].append(p.rect(x=[480],y=[434-diff],width=20,height=40,color='yellow',fill_alpha=0)) # 0
led_h[0].append(p.rect(x=[504],y=[473-diff],width=40,height=20,color='yellow',fill_alpha=0)) # 1
led_h[0].append(p.rect(x=[565],y=[473-diff],width=40,height=20,color='yellow',fill_alpha=0)) # 2
led_h[0].append(p.rect(x=[591],y=[434-diff],width=20,height=40,color='yellow',fill_alpha=0)) # 3
led_h[0].append(p.rect(x=[504],y=[388-diff],width=40,height=20,color='yellow',fill_alpha=0)) # 4
led_h[0].append(p.rect(x=[565],y=[388-diff],width=40,height=20,color='yellow',fill_alpha=0)) # 5
led_h[0].append(p.rect(x=[536],y=[434-diff],width=20,height=40,color='yellow',fill_alpha=0)) # 6

show(p, notebook_handle=True)
led(0,'off')
led(1,'off')
state = [127,127]

Loading BokehJS ...

In [5]:
def toggle_led(side,state,led_num):
    state[side] = state[side] ^ 2**led_num
    led(side,state[side])
    x = led_h[side][led_num]
    if x.glyph.fill_alpha == 0:
        x.glyph.fill_alpha = 1
    else:
        x.glyph.fill_alpha = 0
    push_notebook()
    print(state)


In [ ]:
toggle_led(0,state,6)

In [ ]:
x = led_h[0][3]
x.glyph.fill_alpha = 1
push_notebook()

In [8]:
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Toggle

def test_fun():
    print('hello')
    toggle.label = 'Potato'
    push_notebook()

toggle = Toggle(label="Foo", button_type="default")
toggle.on_click(test_fun)

#show(widgetbox(toggle), notebook_handle=True)

In [ ]:
play = True

def play_pause(arg):
    push_notebook()
    toggle.

toggle.on_click(play_pause)


In [ ]:
toggle.label = 'Potato'
push_notebook()

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

a = widgets.IntSlider()
b = widgets.IntSlider()
c = widgets.IntSlider()
ui = widgets.HBox([a, b, c])
def f(a, b, c):
    print((a, b, c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

In [9]:
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Toggle

output_file("toggle.html")

toggle = Toggle(label="Foo", button_type="success")

show(widgetbox(toggle))